<a href="https://colab.research.google.com/github/thevirusoup/thevirusoup/blob/main/Waste_to_Cash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from flask import Flask, request, jsonify
import tensorflow as tf
import numpy as np
import cv2
import firebase_admin
from firebase_admin import credentials, firestore

app = Flask(__name__)

# Load pre-trained AI waste classification model
model = tf.keras.models.load_model('waste_classifier.h5')
class_names = ['Recyclable', 'Biodegradable', 'Hazardous']

# Initialize Firebase
cred = credentials.Certificate("firebase_credentials.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

@app.route('/classify', methods=['POST'])
def classify_waste():
    file = request.files['image']
    image = cv2.imdecode(np.frombuffer(file.read(), np.uint8), cv2.IMREAD_COLOR)
    image = cv2.resize(image, (224, 224)) / 255.0
    image = np.expand_dims(image, axis=0)

    prediction = model.predict(image)
    category = class_names[np.argmax(prediction)]

    return jsonify({'category': category})

@app.route('/update_points', methods=['POST'])
def update_points():
    data = request.json
    user_id = data.get('user_id')
    category = data.get('category')
    points = {'Recyclable': 10, 'Biodegradable': 5, 'Hazardous': 0}.get(category, 0)

    user_ref = db.collection('users').document(user_id)
    user_doc = user_ref.get()
    if user_doc.exists:
        user_ref.update({'points': firestore.Increment(points)})
    else:
        user_ref.set({'points': points})

    return jsonify({'message': 'Points updated', 'points_awarded': points})

@app.route('/leaderboard', methods=['GET'])
def leaderboard():
    users = db.collection('users').order_by('points', direction=firestore.Query.DESCENDING).limit(10).stream()
    leaderboard_data = [{"user_id": user.id, "points": user.to_dict().get("points", 0)} for user in users]

    return jsonify({'leaderboard': leaderboard_data})

if __name__ == '__main__':
    app.run(debug=True)


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'waste_classifier.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)